In [21]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from idtrackerai import ListOfBlobs
from matplotlib.patches import Ellipse
from scipy.interpolate import splev

from fishmidline import get_spline

list_of_blobs = ListOfBlobs.load("100_3.pickle")



Output()

In [24]:
n_points=3
midline= np.zeros((19526,100, n_points, 2), dtype=np.float32)
for IDENTITY in range(1,101):
    print(f"Processing IDENTITY={IDENTITY}")
    for frame_number in range(19526):
        blobs_in_frame = list_of_blobs.blobs_in_video[frame_number]
        try:
            blob = [blob for blob in blobs_in_frame if IDENTITY in blob.final_identities][0]
            try:
                spline_params = get_spline(blob)
            except TypeError:
                print(f"Skipping IDENTITY={IDENTITY} in frame {frame_number} due to TypeError")
                continue
            spline_samples = np.linspace(0, 1, n_points)
            midline_x, midline_y = splev(spline_samples, spline_params)
            head = splev(0, spline_params)
            derivative_x, derivative_y = splev(spline_samples, spline_params, der=1)
            midline_angles = np.arctan2(derivative_x, derivative_y)

            #center, (width, height), angle = cv2.fitEllipse(blob.contour)
            #center = np.asarray(center) - blob.bbox_in_frame_coordinates[0] + 1
            midline[frame_number,IDENTITY-1, :, 0] = midline_x+blob.bbox_in_frame_coordinates[0][0]
            midline[frame_number,IDENTITY-1, :, 1] = midline_y+blob.bbox_in_frame_coordinates[0][1]
        except IndexError:
            print(f"{IDENTITY=} in {frame_number=} not found!")
            continue

Processing IDENTITY=1


/tmp/ipykernel_168194/4289584096.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) bbox_in_frame_coordinates. (Use `bbox_corners` instead) -- Deprecated since version 6.0.0.
  midline[frame_number,IDENTITY-1, :, 0] = midline_x+blob.bbox_in_frame_coordinates[0][0]
/tmp/ipykernel_168194/4289584096.py:23: DeprecationWarning: Call to deprecated function (or staticmethod) bbox_in_frame_coordinates. (Use `bbox_corners` instead) -- Deprecated since version 6.0.0.
  midline[frame_number,IDENTITY-1, :, 1] = midline_y+blob.bbox_in_frame_coordinates[0][1]


Processing IDENTITY=2
Skipping IDENTITY=2 in frame 1386 due to TypeError
Processing IDENTITY=3
Processing IDENTITY=4
Processing IDENTITY=5
Processing IDENTITY=6
Processing IDENTITY=7
Processing IDENTITY=8
Processing IDENTITY=9
Processing IDENTITY=10
Processing IDENTITY=11
Processing IDENTITY=12
Processing IDENTITY=13
Processing IDENTITY=14
Processing IDENTITY=15
Processing IDENTITY=16
Processing IDENTITY=17
Processing IDENTITY=18
Processing IDENTITY=19
Processing IDENTITY=20
Processing IDENTITY=21
Processing IDENTITY=22
Processing IDENTITY=23
Processing IDENTITY=24
Processing IDENTITY=25
Skipping IDENTITY=25 in frame 13686 due to TypeError
Processing IDENTITY=26
Processing IDENTITY=27
Processing IDENTITY=28
Processing IDENTITY=29
Processing IDENTITY=30
Processing IDENTITY=31
Processing IDENTITY=32
Processing IDENTITY=33
Skipping IDENTITY=33 in frame 1299 due to TypeError
Processing IDENTITY=34
Processing IDENTITY=35
Processing IDENTITY=36
Processing IDENTITY=37
Processing IDENTITY=38
P

/mnt/c/zebrafish1/fishmidline.py:71: RuntimeWarning: invalid value encountered in divide
  return (dx_1 * dy_2 - dy_1 * dx_2) / np.power(dx_1 * dx_1 + dy_1 * dy_1, 3 / 2)


Processing IDENTITY=83
Processing IDENTITY=84
Processing IDENTITY=85
Skipping IDENTITY=85 in frame 585 due to TypeError
Processing IDENTITY=86
Processing IDENTITY=87
Skipping IDENTITY=87 in frame 585 due to TypeError
Processing IDENTITY=88
Processing IDENTITY=89
Skipping IDENTITY=89 in frame 3524 due to TypeError
Processing IDENTITY=90
Processing IDENTITY=91
Processing IDENTITY=92
Processing IDENTITY=93
Processing IDENTITY=94
Processing IDENTITY=95
Skipping IDENTITY=95 in frame 3113 due to TypeError
Processing IDENTITY=96
Skipping IDENTITY=96 in frame 13321 due to TypeError
Processing IDENTITY=97
Processing IDENTITY=98
Processing IDENTITY=99
Processing IDENTITY=100


In [25]:
import numpy as np
from scipy.interpolate import interp1d

# Example data: (N, 2) array with some (0, 0) entries

# Step 1: Identify valid (non-zero) entries
for i in range(midline.shape[1]):
    for j in range(3):
        validx= midline[:, i, j, 0] != 0  # Mask for valid entries (non-zero x-coordinates)
        validy = midline[:, i, j, 1] != 0  # Mask for valid entries (non-zero y-coordinates)
        invalidx= midline[:, i, j, 0] == 0  # Mask for invalid entries (zero x-coordinates)
        invalidy = midline[:, i, j, 1] == 0  # Mask for invalid entries (zero y-coordinates)
        interp_func_x = interp1d(
            np.arange(midline.shape[0])[validx], 
            midline[validx, i, j, 0],
            kind='linear', 
            fill_value='extrapolate'
        )
        interp_func_y = interp1d(
            np.arange(midline.shape[0])[validy], 
            midline[validy, i, j, 1],
            kind='linear', 
            fill_value='extrapolate'
        )
        midline[invalidx, i, j, 0] = interp_func_x(np.arange(midline.shape[0])[invalidx])
        midline[invalidy, i, j, 1] = interp_func_y(np.arange(midline.shape[0])[invalidy])


In [26]:
np.save("data_midline/100_3.npy", midline)

In [ ]:
midline=np.load('data_midline/100_2.npy')

def ensure_consistent_head_tail(fish_data):
    """
    fish_data: numpy array of shape (N, m, k, 2), where k=3 (head, body, tail).
    Modifies the array in-place to ensure the head is always moving forward.
    """
    N, m, k, _ = fish_data.shape
    
    for i in range(N - 1):  # Skip last frame (no future data)
            for j in range(m):
                # Current body, head, and tail positions
                curr_body = fish_data[i, j, 1]
                curr_head = fish_data[i, j, 0]
                curr_tail = fish_data[i, j, 2]
                
                # Body velocity (using next frame)
                next_body = fish_data[i+1, j, 1]
                body_velocity = next_body - curr_body
                
                # Vectors from head and tail to body
                head_to_body =  curr_head-curr_body
                tail_to_body =  curr_tail-curr_body 
                
                # Compute angles (in radians)
                def angle_between(v1, v2):
                    return np.arccos(np.clip(
                        np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1
                    ))
                
                angle_head = angle_between(body_velocity, head_to_body)
                angle_tail = angle_between(body_velocity, tail_to_body)
                
                # If head angle is larger, swap head and tail
                if angle_head > angle_tail:
                    fish_data[i, j, 0], fish_data[i, j, 2] = fish_data[i, j, 2].copy(), fish_data[i, j, 0].copy()

    return fish_data
# Ensure consistent head and tail across frames
midline = ensure_consistent_head_tail(midline)
def ensure_consistent_head_tail(fish_data):
    """
    fish_data: numpy array of shape (N, m, k, 2), where k=3 (head, body, tail).
    Modifies the array in-place to ensure consistent head/tail labeling.
    """
    N, m, k, _ = fish_data.shape
    
    for i in range(1, N):
        for j in range(m):
            # Previous head and tail (frame i-1)
            prev_head = fish_data[i-1, j, 0]
            prev_body = fish_data[i-1, j, 1]
            prev_tail = fish_data[i-1, j, 2]
            
            # Current candidate head and tail (frame i)
            curr_head = fish_data[i, j, 0]
            curr_body= fish_data[i, j, 1]
            curr_tail = fish_data[i, j, 2]
            prev_head_to_body =  prev_head-prev_body
            head_to_body =  curr_head-curr_body
            tail_to_body =  curr_tail-curr_body 
            def angle_between(v1, v2):
                return np.arccos(np.clip(
                    np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1
                ))
            # Compute distances
            angle_head = angle_between(prev_head_to_body, head_to_body)
            angle_tail = angle_between(prev_head_to_body, tail_to_body)
            
            # If curr_tail is closer to prev_head, swap head and tail
            if angle_head > angle_tail:
                fish_data[i, j, 0], fish_data[i, j, 2] = fish_data[i, j, 2].copy(), fish_data[i, j, 0].copy()

    return fish_data
midline = ensure_consistent_head_tail(midline)
# Save the modified midline data
np.save("data_midline/100_2_consistent.npy", midline)

/tmp/ipykernel_168194/3046257258.py:28: RuntimeWarning: invalid value encountered in scalar divide
  np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1


: 